# AIR HANDLING UNIT 1 (AHU R50): SUPPLY-SIDE

In this example, an air supply duct system is analyzed. The configuration of the duct network is entirely done by coding, without making use of a network configuration file. The duct sections with fittings are created before the `DuctNetwork` object is created. Afterward, pseudo-conduits are added to close the loops and analyze the network.  

In [1]:
import warnings
from hvac.fluids import CoolPropWarning

warnings.filterwarnings('ignore', category=CoolPropWarning)

In [2]:
from IPython.display import display, HTML

In [3]:
from hvac import Quantity
from hvac.fluid_flow import (
    DuctNetwork,
    Duct,
    Circular,
    Rectangular,
    rectangular_duct_schedule,
    PseudoConduit
)
import hvac.fluid_flow.fittings.duct as duct_fittings
from hvac.fluids import Fluid

In [4]:
Q_ = Quantity

In [5]:
Air = Fluid('Air')
standard_air = Air(
    T=Q_(20, 'degC'),
    P=Q_(101_325, 'Pa')
)

<img src="R50_supply.png" alt="duct layout">

# Duct Sections and Fittings

In [6]:
dps = Q_(1.0, 'Pa/m')  # specific pressure drop
wr = Q_(0.09, 'mm')    # wall roughness of medium smooth ductwork

In [7]:
supply_ducts = []

In [8]:
def add_duct(node_IDs, loop_ID, duct):
    supply_ducts.append({
        'conduit': duct,
        'start_node_ID': node_IDs[0],
        'end_node_ID': node_IDs[1],
        'conduit_ID': f'{node_IDs[0]}-{node_IDs[1]}',
        'loop_ID': loop_ID
    })

## Section 0/1

In [9]:
duct_0_1 = Duct.create(
    length=Q_(0.1, 'm'),
    wall_roughness=wr,
    fluid=standard_air,
    cross_section=Rectangular.create(
        height=Q_(700, 'mm'),
        width=Q_(1400, 'mm')
    ),
    volume_flow_rate=Q_(7500.0, 'm ** 3 / hr')
)

In [10]:
print(
    f"air velocity: {duct_0_1.velocity.to('m / s'):~P.2f}\n"
    f"pressure loss: {duct_0_1.pressure_drop.to('Pa'):~P.3f}"
)

air velocity: 2.13 m/s
pressure loss: 0.005 Pa


## Section 1/2

In [11]:
duct_1_2 = Duct.create(
    length=Q_(2.0, 'm'),
    wall_roughness=wr,
    fluid=standard_air,
    cross_section=Rectangular.create(
        height=Q_(300, 'mm'),
        width=Q_(1200, 'mm')
    ),
    volume_flow_rate=Q_(-7500.0, 'm ** 3 / hr')
)

add_duct(
    node_IDs=(1, 2),
    loop_ID='L1',
    duct=duct_1_2
)

In [12]:
print(
    f"duct width: {duct_1_2.cross_section.width.to('mm'):~P.0f}\n"
    f"air velocity: {duct_1_2.velocity.to('m / s'):~P.2f}\n"
    f"pressure loss: {duct_1_2.pressure_drop.to('Pa'):~P.3f}"
)

duct width: 1200 mm
air velocity: 5.79 m/s
pressure loss: 1.447 Pa


### Reducer

In [13]:
# 1400 x 700 --> 1200 x 300
reducer_1_2 = duct_fittings.ConvergingTransitionA9B(
    duct_small=duct_1_2,
    duct_large=duct_0_1,
    theta=Q_(60.0, 'deg'),
    length=Q_(0.3, 'm')
)

duct_1_2.add_fitting(
    zeta=reducer_1_2.zeta_small,
    ID='RED-1/2'
)

In [14]:
print(
    f"pressure loss: {reducer_1_2.pressure_drop.to('Pa'):~P.3f}"
)

pressure loss: 1.713 Pa


### Elbow 90°

In [15]:
elbow_1_2 = duct_fittings.ElbowA7F(
    duct=duct_1_2,
    radius=Q_(0.5 * duct_1_2.cross_section.width.to('mm').m, 'mm')
)

duct_1_2.add_fitting(
    zeta=elbow_1_2.zeta,
    ID='ELB90-1/2'
)

In [16]:
print(
    f"pressure loss: {elbow_1_2.pressure_drop.to('Pa'):~P.3f}"
)

pressure loss: 30.256 Pa


## Section 2/3

In [17]:
duct_2_3 = Duct.create(
    length=Q_(4.738, 'm'),
    wall_roughness=wr,
    fluid=standard_air,
    cross_section=Rectangular.create(
        height=Q_(300, 'mm'),
        width=Q_(1200, 'mm')
    ),
    volume_flow_rate=Q_(-7500.0, 'm ** 3 / hr'),
)

add_duct(
    node_IDs=(2, 3),
    loop_ID='L1',
    duct=duct_2_3
)

In [18]:
print(
    f"duct width: {duct_2_3.cross_section.width.to('mm'):~P.0f}\n"
    f"air velocity: {duct_2_3.velocity.to('m / s'):~P.2f}\n"
    f"pressure drop: {duct_2_3.pressure_drop.to('Pa'):~P.3f}"
)

duct width: 1200 mm
air velocity: 5.79 m/s
pressure drop: 3.428 Pa


## Section 3/4

In [19]:
duct_3_4 = Duct.create(
    length=Q_(0.1, 'm'),
    wall_roughness=wr,
    fluid=standard_air,
    cross_section=Rectangular.create(
        height=Q_(300.0, 'mm'),
        schedule=rectangular_duct_schedule
    ),
    volume_flow_rate=Q_(-1875.0, 'm ** 3 / hr'),
    specific_pressure_drop=dps
)

add_duct(
    node_IDs=(3, 4),
    loop_ID=('L1', 'L2'),
    duct=duct_3_4
)

In [20]:
print(
    f"duct width: {duct_3_4.cross_section.width.to('mm'):~P.0f}\n"
    f"air velocity: {duct_3_4.velocity.to('m / s'):~P.2f}\n"
    f"pressure loss: {duct_3_4.pressure_drop.to('Pa'):~P.3f}"
)

duct width: 350 mm
air velocity: 4.96 m/s
pressure loss: 0.088 Pa


## Section 3/5

In [21]:
duct_3_5 = Duct.create(
    length=Q_(1.755, 'm'),
    wall_roughness=wr,
    fluid=standard_air,
    cross_section=Rectangular.create(
        height=Q_(300, 'mm'),
        width=duct_2_3.cross_section.width
    ),
    volume_flow_rate=Q_(-(7500 - 1875), 'm ** 3 / hr')
)

add_duct(
    node_IDs=(3, 5),
    loop_ID='L2',
    duct=duct_3_5
)

In [22]:
print(
    f"duct width: {duct_3_5.cross_section.width.to('mm'):~P.0f}\n"
    f"air velocity: {duct_3_5.velocity.to('m / s'):~P.2f}\n"
    f"pressure loss: {duct_3_5.pressure_drop.to('Pa'):~P.3f}"
)

duct width: 1200 mm
air velocity: 4.34 m/s
pressure loss: 0.746 Pa


## Main Section 2/3 to Branch Section 3/4

In [23]:
tee_2_3 = duct_fittings.DivergingJunctionA11P(
    duct_b=duct_3_4,
    duct_c=duct_2_3,
    duct_s=duct_3_5
)

In [24]:
print(
    f"pressure drop straight: {tee_2_3.pressure_drop_main.to('Pa'):~P.3f}\n"
    f"pressure drop branch: {tee_2_3.pressure_drop_branch.to('Pa'):~P.3f}"
)

pressure drop straight: 0.605 Pa
pressure drop branch: 24.910 Pa


In [25]:
duct_2_3.add_fitting(
    zeta=tee_2_3.zeta_c,
    ID='TEE-C-2/3'
)

duct_3_4.add_fitting(
    zeta=tee_2_3.zeta_b,
    ID='TEE-B-2/3'
)

## Section 4/6

In [26]:
duct_4_6 = Duct.create(
    length=Q_(1.106, 'm'),
    wall_roughness=wr,
    fluid=standard_air,
    cross_section=Circular.create(
        nominal_diameter=Q_(250, 'mm')
    ),
    volume_flow_rate=Q_(-1875, 'm ** 3 / hr'),
)

add_duct(
    node_IDs=(4, 6),
    loop_ID=('L1', 'L2'),
    duct=duct_4_6
)

In [27]:
print(
    f"diameter: {duct_4_6.cross_section.equivalent_diameter.to('mm'):~P.0f}\n"
    f"air velocity: {duct_4_6.velocity.to('m / s'):~P.2f}\n"
    f"pressure loss: {duct_4_6.pressure_drop.to('Pa'):~P.3f}"
)

diameter: 250 mm
air velocity: 10.61 m/s
pressure loss: 5.500 Pa


### Reducer

In [28]:
# 1200 x 300 --> 250
reducer_4_6 = duct_fittings.ConvergingTransitionA9C(
    duct_small=duct_4_6
)

duct_4_6.add_fitting(
    zeta=reducer_4_6.zeta_small,
    ID='RED-4/6'
)

In [29]:
print(
    f"pressure loss: {reducer_4_6.pressure_drop.to('Pa'):~P.3f}"
)

pressure loss: 3.875 Pa


### Exit

In [30]:
exit_4_6 = duct_fittings.ExitA13J(
    duct=duct_4_6
)

duct_4_6.add_fitting(
    zeta=exit_4_6.zeta,
    ID='EXIT-4/6'
)

In [31]:
print(
    f"pressure drop: {exit_4_6.pressure_drop.to('Pa'):~P.3f}"
)

pressure drop: 67.805 Pa


## Section 5/7

In [32]:
duct_5_7 = Duct.create(
    length=Q_(0.1, 'm'),
    wall_roughness=wr,
    fluid=standard_air,
    cross_section=Rectangular.create(
        height=Q_(300.0, 'mm'),
        schedule=rectangular_duct_schedule
    ),
    volume_flow_rate=Q_(-1875.0, 'm ** 3 / hr'),
    specific_pressure_drop=dps
)

add_duct(
    node_IDs=(5, 7),
    loop_ID=('L2', 'L3'),
    duct=duct_5_7
)

In [33]:
print(
    f"duct width: {duct_5_7.cross_section.width.to('mm'):~P.0f}\n"
    f"air velocity: {duct_5_7.velocity.to('m / s'):~P.2f}\n"
    f"pressure drop: {duct_5_7.pressure_drop.to('Pa'):~P.3f}"
)

duct width: 350 mm
air velocity: 4.96 m/s
pressure drop: 0.088 Pa


## Section 5/8

In [34]:
duct_5_8 = Duct.create(
    length=Q_(0.877, 'm'),
    wall_roughness=wr,
    fluid=standard_air,
    cross_section=Rectangular.create(
        height=Q_(300, 'mm'),
        width=duct_3_5.cross_section.width
    ),
    volume_flow_rate=Q_(-(7500 - 2 * 1875), 'm ** 3 / hr'),
)

add_duct(
    node_IDs=(5, 8),
    loop_ID='L3',
    duct=duct_5_8
)

In [35]:
print(
    f"duct width: {duct_5_8.cross_section.width.to('mm'):~P.0f}\n"
    f"air velocity: {duct_5_8.velocity.to('m / s'):~P.2f}\n"
    f"pressure drop: {duct_5_8.pressure_drop.to('Pa'):~P.3f}"
)

duct width: 1200 mm
air velocity: 2.89 m/s
pressure drop: 0.177 Pa


## Main Section 3/5 to Branch Section 5/7

In [36]:
tee_3_5 = duct_fittings.DivergingJunctionA11P(
    duct_b=duct_5_7,
    duct_c=duct_3_5,
    duct_s=duct_5_8
)

duct_3_5.add_fitting(
    zeta=tee_3_5.zeta_c,
    ID='TEE-C-3/5'
)

duct_5_7.add_fitting(
    zeta=tee_3_5.zeta_b,
    ID='TEE-B-5/7'
)

In [37]:
print(
    f"pressure drop straight: {tee_3_5.pressure_drop_main.to('Pa'):~P.3f}\n"
    f"pressure drop branch: {tee_3_5.pressure_drop_branch.to('Pa'):~P.3f}"
)

pressure drop straight: 0.529 Pa
pressure drop branch: 18.514 Pa


## Section 7/9

In [38]:
duct_7_9 = Duct.create(
    length=Q_(1.106, 'm'),
    wall_roughness=wr,
    fluid=standard_air,
    cross_section=Circular.create(
        nominal_diameter=Q_(250, 'mm')
    ),
    volume_flow_rate=Q_(-1875, 'm ** 3 / hr')
)

add_duct(
    node_IDs=(7, 9),
    loop_ID=('L2', 'L3'),
    duct=duct_7_9
)

In [39]:
print(
    f"diameter: {duct_7_9.cross_section.equivalent_diameter.to('mm'):~P.0f}\n"
    f"air velocity: {duct_7_9.velocity.to('m / s'):~P.2f}\n"
    f"pressure loss: {duct_7_9.pressure_drop.to('Pa'):~P.3f}"
)

diameter: 250 mm
air velocity: 10.61 m/s
pressure loss: 5.500 Pa


### Reducer

In [40]:
# 1200 x 300 --> 250
reducer_7_9 = duct_fittings.ConvergingTransitionA9C(
    duct_small=duct_7_9
)

duct_7_9.add_fitting(
    zeta=reducer_7_9.zeta_small,
    ID='RED-7/9'
)

In [41]:
print(
    f"pressure loss: {reducer_7_9.pressure_drop.to('Pa'):~P.3f}"
)

pressure loss: 3.875 Pa


### Exit

In [42]:
exit_7_9 = duct_fittings.ExitA13J(
    duct=duct_7_9
)

duct_7_9.add_fitting(
    zeta=exit_7_9.zeta,
    ID='EXIT-7/9'
)

In [43]:
print(
    f"pressure drop: {exit_7_9.pressure_drop.to('Pa'):~P.3f}"
)

pressure drop: 67.805 Pa


## Section 8/10

In [44]:
duct_8_10 = Duct.create(
    length=Q_(8.414, 'm'),
    wall_roughness=wr,
    fluid=standard_air,
    cross_section=Rectangular.create(
        height=Q_(300, 'mm'),
        width=Q_(1000, 'mm')),
    volume_flow_rate=Q_(-(7500 - 2 * 1875), 'm ** 3 / hr')
)

add_duct(
    node_IDs=(8, 10),
    loop_ID='L3',
    duct=duct_8_10
)

In [45]:
print(
    f"duct width: {duct_8_10.cross_section.width.to('mm'):~P.0f}\n"
    f"air velocity: {duct_8_10.velocity.to('m / s'):~P.2f}\n"
    f"pressure loss: {duct_8_10.pressure_drop.to('Pa'):~P.3f}"
)

duct width: 1000 mm
air velocity: 3.47 m/s
pressure loss: 2.488 Pa


### Reducer

In [46]:
# 1200 x 300 --> 1000 x 300
reducer_8_10 = duct_fittings.ConvergingTransitionA9B(
    duct_small=duct_8_10,
    duct_large=duct_5_8,
    theta=Q_(60.0, 'deg'),
    length=Q_(0.3, 'm')
)

duct_8_10.add_fitting(
    zeta=reducer_8_10.zeta_small,
    ID='RED-8/10'
)

In [47]:
print(
    f"pressure loss: {reducer_8_10.pressure_drop.to('Pa'):~P.3f}"
)

pressure loss: 0.227 Pa


## Section 10/11

In [48]:
duct_10_11 = Duct.create(
    length=Q_(0.1, 'm'),
    wall_roughness=wr,
    fluid=standard_air,
    cross_section=Rectangular.create(
        height=Q_(300.0, 'mm'),
        width=Q_(300, 'mm')
    ),
    volume_flow_rate=Q_(-1875.0, 'm ** 3 / hr')
)

add_duct(
    node_IDs=(10, 11),
    loop_ID=('L3', 'L4'),
    duct=duct_10_11
)

In [49]:
print(
    f"duct width: {duct_10_11.cross_section.width.to('mm'):~P.0f}\n"
    f"air velocity: {duct_10_11.velocity.to('m / s'):~P.2f}\n"
    f"pressure loss: {duct_10_11.pressure_drop.to('Pa'):~P.3f}"
)

duct width: 300 mm
air velocity: 5.79 m/s
pressure loss: 0.128 Pa


## Section 10/13

In [50]:
duct_10_13 = Duct.create(
    length=Q_(1.085, 'm'),
    wall_roughness=wr,
    fluid=standard_air,
    cross_section=Rectangular.create(
        height=Q_(300, 'mm'),
        width=duct_8_10.cross_section.width
    ),
    volume_flow_rate=Q_(-1875, 'm ** 3 / hr')
)

add_duct(
    node_IDs=(10, 13),
    loop_ID='L4',
    duct=duct_10_13
)

In [51]:
print(
    f"duct width: {duct_10_13.cross_section.width.to('mm'):~P.0f}\n"
    f"air velocity: {duct_10_13.velocity.to('m / s'):~P.2f}\n"
    f"pressure loss: {duct_10_13.pressure_drop.to('Pa'):~P.3f}"
)

duct width: 1000 mm
air velocity: 1.74 m/s
pressure loss: 0.091 Pa


### Elbow 90°

In [52]:
elbow_10_13 = duct_fittings.ElbowA7F(
    duct=duct_10_13,
    radius=Q_(0.5 * duct_10_13.cross_section.width.to('mm').m, 'mm')
)

duct_10_13.add_fitting(
    zeta=elbow_10_13.zeta,
    ID='ELB90-10/13'
)

In [53]:
print(
    f"pressure loss: {elbow_10_13.pressure_drop.to('Pa'):~P.3f}"
)

pressure loss: 2.975 Pa


## Main Section 8/10 to Branch Section 10/11

In [54]:
tee_8_10 = duct_fittings.DivergingJunctionA11P(
    duct_b=duct_10_11,
    duct_c=duct_8_10,
    duct_s=duct_10_13
)

duct_8_10.add_fitting(
    zeta=tee_8_10.zeta_c,
    ID='TEE-C-8/10'
)

duct_10_11.add_fitting(
    zeta=tee_8_10.zeta_b,
    ID='TEE-B-8/10'
)

In [55]:
print(
    f"pressure loss straight: {tee_8_10.pressure_drop_main.to('Pa'):~P.3f}\n"
    f"pressure loss branch: {tee_8_10.pressure_drop_branch.to('Pa'):~P.3f}"
)

pressure loss straight: 0.653 Pa
pressure loss branch: 23.477 Pa


## Section 11/12

In [56]:
duct_11_12 = Duct.create(
    length=Q_(1.106, 'm'),
    wall_roughness=wr,
    fluid=standard_air,
    cross_section=Circular.create(
        nominal_diameter=Q_(250, 'mm')
    ),
    volume_flow_rate=Q_(-1875, 'm ** 3 / hr'),
)

add_duct(
    node_IDs=(11, 12),
    loop_ID=('L3', 'L4'),
    duct=duct_11_12
)

In [57]:
print(
    f"duct diameter: {duct_11_12.cross_section.equivalent_diameter.to('mm'):~P.0f}\n"
    f"air velocity: {duct_11_12.velocity.to('m / s'):~P.2f}\n"
    f"pressure loss: {duct_11_12.pressure_drop.to('Pa'):~P.3f}"
)

duct diameter: 250 mm
air velocity: 10.61 m/s
pressure loss: 5.500 Pa


### Reducer

In [58]:
#  (600 x 300) --> 250
reducer_11_12 = duct_fittings.ConvergingTransitionA9C(
    duct_small=duct_11_12
)

duct_11_12.add_fitting(
    zeta=reducer_11_12.zeta_small,
    ID='RED-11/12'
)

In [59]:
print(f"pressure loss: {reducer_11_12.pressure_drop.to('Pa'):~P.3f}")

pressure loss: 3.875 Pa


### Exit

In [60]:
exit_11_12 = duct_fittings.ExitA13J(
    duct=duct_11_12
)

duct_11_12.add_fitting(
    zeta=exit_11_12.zeta,
    ID='EXIT-11/12'
)

In [61]:
print(
    f"pressure loss: {exit_11_12.pressure_drop.to('Pa'):~P.3f}"
)

pressure loss: 67.805 Pa


## Section 13/14

In [62]:
duct_13_14 = Duct.create(
    length=Q_(1.106, 'm'),
    wall_roughness=wr,
    fluid=standard_air,
    cross_section=Circular.create(
        nominal_diameter=Q_(250, 'mm')
    ),
    volume_flow_rate=Q_(-1875, 'm ** 3 / hr'),
)

add_duct(
    node_IDs=(13, 14),
    loop_ID='L4',
    duct=duct_13_14
)

In [63]:
print(
    f"diameter: {duct_13_14.cross_section.equivalent_diameter.to('mm'):~P.0f}\n"
    f"air velocity: {duct_13_14.velocity.to('m / s'):~P.2f}\n"
    f"pressure loss: {duct_13_14.pressure_drop.to('Pa'):~P.3f}"
)

diameter: 250 mm
air velocity: 10.61 m/s
pressure loss: 5.500 Pa


### Reducer

In [64]:
#  (600 x 300) --> 250
reducer_13_14 = duct_fittings.ConvergingTransitionA9C(
    duct_small=duct_13_14
)

duct_13_14.add_fitting(
    zeta=reducer_13_14.zeta_small,
    ID='RED-13/14'
)

In [65]:
print(
    f"pressure loss: {reducer_13_14.pressure_drop.to('Pa'):~P.3f}"
)

pressure loss: 3.875 Pa


### Exit

In [66]:
exit_13_14 = duct_fittings.ExitA13J(
    duct=duct_13_14
)

duct_13_14.add_fitting(
    zeta=exit_13_14.zeta,
    ID='EXIT-13/14'
)

In [67]:
print(
    f"pressure loss: {exit_13_14.pressure_drop.to('Pa'):~P.3f}"
)

pressure loss: 67.805 Pa


# Duct Network

In [68]:
supply_network = DuctNetwork.create(
    ID='supply_unit1',
    fluid=standard_air,
    ambient_air=standard_air,
    wall_roughness=wr,
    start_node_ID=1,
    units={'volume_flow_rate': 'm ** 3 / hr'}
)

for duct in supply_ducts:
    supply_network.add_conduit(**duct)

## Duct Table

In [69]:
duct_table = supply_network.get_duct_table()
display(HTML(duct_table.to_html()))

,duct ID,L [m],Deq [mm],width [mm],height [mm],V [m³/h],v [m/s],Re,Δp-dyn. [Pa]
0,1-2,2.000,620.311168,1200.0,300.0,-7499.995341,5.787033,183791.046592,33.415964
1,2-3,4.738,620.311168,1200.0,300.0,-7496.588520,5.784405,183707.560781,4.031174
2,3-4,0.100,353.963373,350.0,300.0,-1874.947868,4.960180,106030.413859,24.997717
3,3-5,1.755,620.311168,1200.0,300.0,-5624.539588,4.339923,137832.087960,1.275078
4,4-6,1.106,250.000000,NaN,NaN,-1874.996365,10.610309,175507.290179,77.179310
5,5-7,0.100,353.963373,350.0,300.0,-1874.947867,4.960180,106030.413799,18.602416
6,5-8,0.877,620.311168,1200.0,300.0,-3749.843013,2.893397,91891.733345,0.177120
7,7-9,1.106,250.000000,NaN,NaN,-1874.996365,10.610309,175507.290179,77.179310
8,8-10,8.414,573.664927,1000.0,300.0,-3749.360447,3.471630,106015.278247,3.367945
9,10-11,0.100,327.949602,300.0,300.0,-1874.955808,5.786901,114866.768091,23.605493


## Fitting Table

In [70]:
fitting_table = supply_network.get_fitting_table()
display(HTML(fitting_table.to_html()))

,conduit ID,fitting ID,zeta,pressure drop [Pa]
0,1-2,RED-1/2,0.084931,1.713090
1,1-2,ELB90-1/2,1.5,30.255697
2,2-3,TEE-C-2/3,0.029999999999999985,0.604564
3,3-4,TEE-B-2/3,1.681006,24.909672
4,3-5,TEE-C-3/5,0.046666666666666676,0.529389
5,4-6,RED-4/6,0.05714465872332772,3.874678
6,4-6,EXIT-4/6,1.0,67.804728
7,5-7,TEE-B-5/7,1.249425,18.514371
8,7-9,RED-7/9,0.05714465872332772,3.874678
9,7-9,EXIT-7/9,1.0,67.804728


## Flow Paths

In [71]:
flow_path_table = supply_network.get_flow_path_table()
display(HTML(flow_path_table.to_html()))

,path,Δp-elev. [Pa],Δp-dyn. [Pa],Δp-tot. [Pa],Δp-deficit [Pa]
0,1-2|2-3|3-4|4-6,0.0,139.624164,139.624164,3.427919
1,1-2|2-3|3-5|5-7|7-9,0.0,134.503941,134.503941,8.548142
2,1-2|2-3|3-5|5-8|8-10|10-11|11-12,0.0,143.052083,143.052083,0.000000
3,1-2|2-3|3-5|5-8|8-10|10-13|13-14,0.0,122.513022,122.513022,20.539062


# Analysis

## Pseudo-Conduits

Before we can run an analysis on the duct network, we have to close the loops of the network with pseudo-conduits. Pseudo-conduits connect the exits of the network at nodes 14, 12, 9 and 6. These pseudo-conduits close the loops L4 (between node 14 and node 12), L3 (between node 12 and node 9), and L2 (between node 9 and node 6). If we consider that all exits of the network have the same pressure (atmospheric pressure), the pressure difference across a pseudo-conduit connecting two exits is zero. Loop L1 is not being closed yet. Node 1 is the start node of the network where air enters the duct system. We can consider exit node 6 (connected to the other exit nodes by pseudo-conduits and all exit nodes having the same pressure) to be the end node of our duct network. For air to flow from network entry node 1 to the exit nodes of the network (or to end node 6 of the network), entry node 1 must be at higher pressure than the exit nodes (or end node 6 of the network). From the flow table above it follows that for the design volume flow rates, the critical path of the duct network has a total pressure loss of around 143 Pa. So, we can say that the pressure at entry node 1 must be around 143 Pa (gage pressure) with respect to the exit nodes of the network (or end node 6) which are at 0 Pa (gage pressure). This means there is a fixed pressure difference of 143 Pa across the pseudo-conduit of loop L1 that connects start node 1 with end node 6. This value must be assigned a positive sign because air would flow through the pseudo-conduit from node 1 to node 6 which corresponds with the positive loop sense (i.e., the clock-wise sense). 

In [72]:
supply_network.add_conduit(
    conduit=PseudoConduit.create(fixed_pressure_drop=Q_(143, 'Pa')),
    conduit_ID='1-6',
    start_node_ID=1,
    end_node_ID=6,
    loop_ID='L1'
)

supply_network.add_conduit(
    conduit=PseudoConduit.create(fixed_pressure_drop=Q_(0, 'Pa')),
    conduit_ID='9-6',
    start_node_ID=9,
    end_node_ID=6,
    loop_ID='L2'
)

supply_network.add_conduit(
    conduit=PseudoConduit.create(fixed_pressure_drop=Q_(0, 'Pa')),
    conduit_ID='12-9',
    start_node_ID=12,
    end_node_ID=9,
    loop_ID='L3'
)

supply_network.add_conduit(
    conduit=PseudoConduit.create(fixed_pressure_drop=Q_(0, 'Pa')),
    conduit_ID='14-12',
    start_node_ID=14,
    end_node_ID=12,
    loop_ID='L3'
)

## Run Analysis

In [73]:
supply_network.analyze(tolerance=Q_(0.05, 'Pa'))

55

## Results

By looking at the flow paths, we can check the exactness of the results. Each flow path in the duct network should have (approximately) the same total pressure difference, and the pressure difference deficit of each flow path should (approximately) be zero. By taking the tolerance smaller (and possibly increasing the maximum number of iterations), a better precision can be achieved.   

In [74]:
flow_path_table = supply_network.get_flow_path_table()
display(HTML(flow_path_table.to_html()))

,path,Δp-elev. [Pa],Δp-dyn. [Pa],Δp-tot. [Pa],Δp-deficit [Pa]
0,1-2|2-3|3-4|4-6,0.0,142.967247,142.967247,0.000000
1,1-2|2-3|3-5|5-7|7-9,0.0,142.958019,142.958019,0.009228
2,1-2|2-3|3-5|5-8|8-10|10-11|11-12,0.0,142.913259,142.913259,0.053988
3,1-2|2-3|3-5|5-8|8-10|10-13|13-14,0.0,142.909694,142.909694,0.057553


In the duct table, we can look at the volume flow rates through the exit sections of the duct network to check the flow balance of the duct network. 

In [75]:
duct_table = supply_network.get_duct_table()
display(HTML(duct_table.to_html()))

,duct ID,L [m],Deq [mm],width [mm],height [mm],V [m³/h],v [m/s],Re,Δp-dyn. [Pa]
0,1-2,2.000,620.311168,1200.0,300.0,-7738.948287,5.971411,189646.705158,35.572347
1,2-3,4.738,620.311168,1200.0,300.0,-7668.367230,5.916950,187917.081916,4.206503
2,3-4,0.100,353.963373,350.0,300.0,-1884.384740,4.985145,106564.079577,25.249915
3,3-5,1.755,620.311168,1200.0,300.0,-5837.871701,4.504531,143059.895509,1.368996
4,4-6,1.106,250.000000,NaN,NaN,-1884.233497,10.662580,176371.923251,77.938482
5,5-7,0.100,353.963373,350.0,300.0,-1933.411733,5.114846,109336.611258,19.780169
6,5-8,0.877,620.311168,1200.0,300.0,-3912.467352,3.018879,95876.922133,0.191397
7,7-9,1.106,250.000000,NaN,NaN,-1933.261128,10.940020,180961.109057,82.030004
8,8-10,8.414,573.664927,1000.0,300.0,-3890.323319,3.602151,110001.082865,3.609528
9,10-11,0.100,327.949602,300.0,300.0,-1848.602839,5.705564,113252.287172,22.946854
